In [1]:
import random
import math

In [13]:
# Class Point
class Point:
    def __init__(self, x, y, color=None):
        self.x = x
        self.y = y
        self.color = color
    
    def __str__(self):
        return "Point: (" + str(self.x) + ", " + str(self.y) + ")" + " | " + str(self.color)

In [15]:
# Class Colored Vertex (3-color graph coloring)
class Vertex(Point):
    def __init__(self, x, y, color=None, vertex_type=None):
        super().__init__(x, y, color)
        self.vertex_type = vertex_type
    
    def __str__(self):
        return super().__str__() + " | " + str(self.vertex_type)

In [6]:
# Class for Linked Structure of Vertex implemented as doubly linked list
class LinkedNode:
    def __init__(self, vertex):
        self.data = vertex
        self.prev = None
        self.next = None
        
    def __str__(self):
        return self.data.__str__()

In [82]:
# Class Doubly Linked List - Representation for Convex, Reflex and Ear Nodes of the Polygon
class DoubleLinkedList:
    def __init__(self, cycle=False):
        self.head = None
        self.tail = None
        self.cycle = cycle
        
    def add_node(self, node):
        if self.head == None:
            self.head = node
            self.tail = node
        else:
            # insert at the end
            node.prev = self.tail
            self.tail.next = node
            self.tail = node
        
        # if cyclic    
        if self.cycle:
            self.head.prev = self.tail
            self.tail.next = self.head
            
    def list_nodes(self):
        current = self.head
        while True:
            print(current)
            current = current.next
            if current == self.tail.next:
                break
    
    def __iter__(self):
        current = self.head
        while True:
            yield current
            current = current.next
            if current == self.tail.next:
                break

In [102]:
# Class Polygon
class Polygon:
    def __init__(self, vertices):
        # vertices object is a double linked list structure of Vertex Linked Node
        self.vertices = vertices
        
        # create linked lists for convex, reflex, ear vertices
        self.convex_vertices, self.reflex_vertices = self.get_convex_and_reflex_vertices()
        self.ear_vertices = self.get_ear_vertices()
        
        
    def get_convex_and_reflex_vertices(self):
        convex_vertices = DoubleLinkedList()
        reflex_vertices = DoubleLinkedList()
        for vertex in self.vertices:
            if self.is_convex(vertex.prev, vertex, vertex.next):
                convex_vertices.add_node(vertex)
                vertex.data.vertex_type = "convex"
            else:
                reflex_vertices.add_node(vertex)
                vertex.data.vertex_type = "reflex"
                
        return convex_vertices, reflex_vertices

    
    def get_ear_vertices(self):
        ear_vertices = DoubleLinkedList()
        for vertex in self.vertices:
            if self.is_ear(vertex, vertex.prev, vertex.next):
                ear_vertices.add_node(vertex)
                vertex.data.vertex_type = "ear"
        return ear_vertices
    
    def is_convex(self, vertex1, vertex2, vertex3):
        angle = calculate_angle(vertex1.data, vertex2.data, vertex3.data)
        print(vertex1)
        print(vertex2)
        print(vertex3)
        print(angle)
        return angle <= math.pi
        
        
    def is_ear(self, vertex, prev_vertex, next_vertex):
        # cannot be ear, if vertex is reflex
        if vertex.data.vertex_type == "reflex":
            return False
        
        triangle = (vertex.data, prev_vertex.data, next_vertex.data)
        for reflex_node in self.reflex_vertices:
            if is_within_triangle(triangle, reflex_node.data):
                return False
        return True
    
    def print_convex_vertices(self):
        self.convex_vertices.list_nodes()
    
    def print_reflex_vertices(self):
        self.reflex_vertices.list_nodes()
        
    def print_ear_vertices(self):
        self.ear_vertices.list_nodes()

In [103]:
# Utility Functions
def is_within_triangle(triangle, vertex):
        triangle_area = calculate_area(triangle[0], triangle[1], triangle[2])
        sub_area_1 = calculate_area(triangle[0], triangle[1], vertex)
        sub_area_2 = calculate_area(triangle[0], triangle[2], vertex)
        sub_area_3 = calculate_area(triangle[1], triangle[2], vertex)
        
        if triangle_area == sub_area_1 + sub_area_2 + sub_area_3:
            return True
        
        return False
        
def calculate_area(vertex1, vertex2, vertex3):
    return abs((vertex1.x * (vertex2.y - vertex3.y) + vertex2.x * (vertex3.y - vertex1.y)
                + vertex3.x * (vertex1.y - vertex2.y)) / 2.0)

def calculate_angle(vertex1, vertex2, vertex3):
    
    # angle of line joining vertex1 and vertex2
    if (vertex2.x - vertex1.x) == 0:
        angle_1 = math.pi/2
    else:
        angle_1 = math.atan((vertex2.y - vertex1.y)/(vertex2.x - vertex1.x))
    
    # angle of line joining vertex2 and vertex3
    if (vertex3.x - vertex2.x) == 0:
        angle_2 = math.pi/2
    else:
        angle_2 = math.atan((vertex3.y - vertex2.y)/(vertex3.x - vertex2.x))
    
    if (angle_1 - angle_2) > 0:
        return 2*math.pi - (angle_1 - angle_2)
    
    return (angle_2 - angle_1)

In [117]:
# create vertices
vertices = [
    Vertex(0,0),
    Vertex(1,1),
    Vertex(2,4),
    Vertex(2,2),
    Vertex(4,4),
    Vertex(6,0),
    Vertex(8,2),
    Vertex(9,-1),
    Vertex(8,-2),
    Vertex(6,-4),
    Vertex(5,-1),
    Vertex(2,-2)
]

linked_vertices = DoubleLinkedList(cycle=True)

for vertex in vertices:
    linked_vertex = LinkedNode(vertex)
    linked_vertices.add_node(linked_vertex)

linked_vertices.list_nodes()

Point: (0, 0) | None | None
Point: (1, 1) | None | None
Point: (2, 4) | None | None
Point: (2, 2) | None | None
Point: (4, 4) | None | None
Point: (6, 0) | None | None
Point: (8, 2) | None | None
Point: (9, -1) | None | None
Point: (8, -2) | None | None
Point: (6, -4) | None | None
Point: (5, -1) | None | None
Point: (2, -2) | None | None


In [115]:
polygon = Polygon(linked_vertices)

In [116]:
polygon.print_convex_vertices()
print("----------------------")
polygon.print_reflex_vertices()
print("----------------------")
polygon.print_ear_vertices()

Point: (0, 0) | None | convex
Point: (1, 1) | None | ear
Point: (6, 0) | None | ear
Point: (9, -1) | None | ear
Point: (8, -2) | None | ear
Point: (5, -1) | None | convex
----------------------
Point: (2, 2) | None | reflex
Point: (4, 4) | None | reflex
Point: (8, 2) | None | reflex
Point: (6, -4) | None | reflex
Point: (2, -2) | None | reflex
----------------------
Point: (1, 1) | None | ear
Point: (6, 0) | None | ear
Point: (9, -1) | None | ear
Point: (8, -2) | None | ear
